In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pickle


In [8]:
with open("output_metrics.pickle", "rb") as f:
    answer = pickle.load(f)


In [9]:
contents = answer[(3, 7)][0].split()
contents

['str',
 'cluster_metrics',
 'metrics',
 'clusters_sizes',
 'weighted_mase',
 'weighted_mape']

Кластеризация

In [20]:
for key, value in answer.items():
    print(f"{value[1]['DB']:.3f} - {key}")

0.550 - (3, 7)
0.577 - (10, 7)
